In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
np.random.seed(1)

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids, headers = load_csv_data(DATA_TRAIN_PATH)


## Do your thing crazy machine learning thing here :) ...

#### Studying the distribution of variables to detect possible categorical or faulty values

In [4]:
from plots import plot_feature_distribution
#fig = plot_feature_distribution(tX, headers, np.arange(len(headers)), "Distribution of features", 5, 6)

In [5]:
#fig.savefig("feature_distribution.png")

As we can see from these plots, different features have different distributions. We can also notice that there seems to be one categorical feature, namely `PRI_jet_num`. This feature is the number of jets used during the experiment. If we look at the documentation of the dataset (https://higgsml.lal.in2p3.fr/files/2014/04/documentation_v1.8.pdf), we can see that some feature are actually affected by the number of jets used.

Features affected by undefined values:
- `DER_mass_MMC` ID=0: undefined if topology of event too far from expected
- `DER_deltaeta_jet_jet` ID=4 : undefined if `PRI_jet_num` <= 1
- `DER_mass_jet_jet` ID=5: undefined if `PRI_jet_num` <= 1
- `DER_prodeta_jet_jet` ID=6: undefined if `PRI_jet_num` <= 1
- `DER_lep_eta_centrality` ID=12: undefined if `PRI_jet_num` <= 1
- `PRI_jet_leading_pt` ID=23: undefined if `PRI_jet_num` == 0
- `PRI_jet_leading_eta` ID=24: undefined if `PRI_jet_num` == 0
- `PRI_jet_leading_phi` ID=25: undefined if `PRI_jet_num` == 0
- `PRI_jet_subleading_pt` ID=26: undefined if `PRI_jet_num` <= 1
- `PRI_jet_subleading_eta` ID=27: undefined if `PRI_jet_num` <= 1
- `PRI_jet_subleading_phi` ID=28: undefined if `PRI_jet_num` <= 1

Hence, it might be a good idea to have estimators for each number of jets (0, 1, and more than 1).
Also, we can see that features `PRI_tau_phi`, `PRI_lep_phi` and `PRI_met_phi` have an almost uniform distribution. Let us look at the distribution of the features that are undefined at some point.

In [23]:
from plots import plot_undefined_features
#plot_undefined_features(tX, headers)

As we can see, there are 5 features (let's call the the `PHI` features) that have a uniform distribution. We can discard those, as they do not bring us any additional information.

In [85]:
for i in np.arange(tX.shape[1]):
    for j in np.arange(i+1, tX.shape[1]):
        t1 = tX[:, i]
        t1 = t1[np.where(t1 > -999)]
        t2 = tX[:, j]
        t2 = t2[np.where(t2 > -999)]
        if t1.shape == t2.shape:
            corr = np.corrcoef(np.c_[t1, t2].T)[0][1]
            if np.abs(corr) > 0.85:
                print(corr, headers[i],i, headers[j], j)

0.9044814595684975 DER_sum_pt 9 PRI_met_sumet 21
0.9656283889164025 DER_sum_pt 9 PRI_jet_all_pt 29
0.8844128574100245 PRI_met_sumet 21 PRI_jet_all_pt 29


As we can see, column `DER_sum_pt` at index 9 is very correlated with 2 columns. We choose to drop it, along with all `PHI` features.

In [24]:
to_delete = [9, 15, 18, 20, 25, 28]
to_keep = [x for x in np.arange(tX.shape[1]) if x not in to_delete]
headers = headers[to_keep]
tX = tX[:, to_keep]
tX[tX == -999] = np.nan
#plot_feature_distribution(tX, headers, np.arange(tX.shape[1]), "Feature distribution", 5, 5)

For now, we will split the dataset into 6 categories:
- `tX_0, y_0` : Features and labels for experiments with 0 jets, that have a defined `DER_mass_MMC`
- `tX_0_nm,, y_0_nm`:  Features and labels for experiments with 0 jets, that have an undefined `DER_mass_MMC`
- `tX_1, y_1` : Features and labels for experiments with 1 jet, that have a defined `DER_mass_MMC`
- `tX_1_nm, y_1_nm`: Features and labels for experiments with 1 jet, that have an undefined `DER_mass_MMC`
- `tX_2, y_2` : Features and labels for experiments with more than 1 jets
- `tX_2_nm, y_2_nm`: Features and labels for experiments with more than 1 jet, that have an undefined `DER_mass_MMC`

In [25]:
from data_processing import split_dataset

jet_column = 18
tX_0, y_0, tX_1, y_1, tX_2, y_2 = split_dataset(tX, y, jet_col=jet_column) # Split into each category

Now that we have split the dataset, we need to select the columns that are meaningful for each category, and add some features using polynomial expansion, exponential, logarithm and so on. For that, we have created a function `enhance_features`. This function adds all the expansions and performs PCA to project the feature matrix on a new basis

In [26]:
def split_train_test(x, y, ratio):
    """ Splits the dataset into train/test data
    
    :param x: Feature matrix
    :param y: Labels
    :param ratio: Ratio for train
    :return: x_train, y_train, x_test, y_test
    """
    # set seed
    indices = np.random.permutation(x.shape[0])  # Get random permutations of the indices
    num_train = int(ratio * x.shape[0])
    train_indices, test_indices = indices[:num_train], indices[num_train:]  # Split indices into train and test
    
    train_x, train_y = x[train_indices], y[train_indices]
    test_x, test_y = x[test_indices], y[test_indices]
    
    return train_x, train_y, test_x, test_y

In [27]:
from data_processing import prepare_for_training, prepare_for_testing
from implementations import reg_logistic_regression, ridge_regression
from cross_validation import cross_validate_degrees

def train_ridge_model(x, y, train_ratio, lambda_):
    x_train, y_train, x_test, y_test = split_train_test(x, y, train_ratio)
    x_train, y_train = prepare_for_training(x_train, y_train, logistic=False)
    x_test, y_test = prepare_for_testing(x_test, y_test, logistic=False)
    
    weights, loss = ridge_regression(y_train, x_train, lambda_)

    score = compute_accuracy(y_test, x_test, weights)
    print(f"Ridge regression got score of {score}, loss of {loss}")
    return weights

def train_logistic_model(x, y, train_ratio, gamma, lambda_, max_iters):
    x_train, y_train, x_test, y_test = split_train_test(x, y, train_ratio)
    x_train, y_train = prepare_for_training(x_train, y_train)
    x_test, y_test = prepare_for_testing(x_test, y_test)

    initial_w = np.zeros((x_train.shape[1], 1))
    weights, loss = reg_logistic_regression(y_train, x_train, lambda_, initial_w, max_iters, gamma)
    
    score = compute_accuracy(y_test, x_test, weights)
    print(f"Regularized logisitc regression got score of {score}, loss of {loss}")
    return weights

#### A few global variable declarations

In [28]:
from feature_expansion import expand_features
train_ratio = 0.9

### Category 0

In [ ]:
fig = cross_validate_degrees(tX_0, y_0, np.logspace(-5, 0, 6), 1e-5, np.arange(1, 10), 4, "Mean accuracy and loss w.r. to polynomial expansion for PRI_jet_num=0", jet_column, logistic=True)

In [31]:
fig.savefig("cross_val_0_log.png")

As we can see, a degree of 3 gives the highest accuracy

In [10]:
#cross_validate_reg(tX_0_exp, y_0, selected_cols_0, 1e-5, np.logspace(-4, 0, 5), 4, "Mean accuracy and Loss w.r. to Regularization for PRI_jet_num=0, DER_MASS_MMC > -999")

In [196]:
degree_0 = 1
tX_0_exp = expand_features(tX_0, degree_0, jet_col=jet_column)
w_0 = train_logistic_model(tX_0_exp, y_0, train_ratio, 1e-5, 0, 1000)#train_ridge_model(tX_0_exp, y_0, train_ratio, 1e-2)#

Performing polynomial expansion up to degree 1
Matrix has now 78 features
Gradient Descent(0/999): loss=62328.487623130844
Gradient Descent(100/999): loss=33728.87433724222
Gradient Descent(200/999): loss=33395.88013009496
Gradient Descent(300/999): loss=33241.93995405746
Gradient Descent(400/999): loss=33142.619210348545
Gradient Descent(500/999): loss=33069.750495989276
Gradient Descent(600/999): loss=33012.19657978584
Gradient Descent(700/999): loss=32964.57270272934
Gradient Descent(800/999): loss=32923.948718661486
Gradient Descent(900/999): loss=32888.57752292921
Regularized logisitc regression got score of 0.8395716573258607, loss of 32857.33386451057


### Category 1

In [ ]:
fig = cross_validate_degrees(tX_1, y_1, np.logspace(-5, 0, 6), 1e-5, np.arange(1, 10), 4, "Mean accuracy and loss w.r. to polynomial expansion for PRI_jet_num=1", jet_column, logistic=True)

In [ ]:
fig.savefig("cross_val_1_log.png")

In [194]:
degree_1 = 1
tX_1_exp = expand_features(tX_1, degree_1, jet_col=jet_column)
w_1 = train_logistic_model(tX_1_exp, y_1, train_ratio, 1e-5, 0, 1300)#train_ridge_model(tX_1_exp, y_1, train_ratio, 1e-3)#

Performing polynomial expansion up to degree 1
Matrix has now 94 features
Gradient Descent(0/1299): loss=48374.04858409802
Gradient Descent(100/1299): loss=33261.26677115651
Gradient Descent(200/1299): loss=32391.19495588421
Gradient Descent(300/1299): loss=32055.827018345677
Gradient Descent(400/1299): loss=31904.26183823479
Gradient Descent(500/1299): loss=31799.195787530756
Gradient Descent(600/1299): loss=31718.68696477509
Gradient Descent(700/1299): loss=31654.65191372418
Gradient Descent(800/1299): loss=31602.35647751285
Gradient Descent(900/1299): loss=31558.800911624283
Gradient Descent(1000/1299): loss=31521.970265731594
Gradient Descent(1100/1299): loss=31490.447306527632
Gradient Descent(1200/1299): loss=31463.19763675362
Regularized logisitc regression got score of 0.7905867182462927, loss of 31439.443292550743


### Category 2

In [ ]:
fig = cross_validate_degrees(tX_2, y_2, np.logspace(-5, 0, 6), 1e-5, np.arange(1, 10), 4, "Mean accuracy and loss w.r. to polynomial expansion for PRI_jet_num=2", jet_column, logistic=True)

In [ ]:
fig.savefig("cross_val_2_log.png")

In [195]:
degree_2 = 1
tX_2_exp = expand_features(tX_2, degree_2, jet_col=jet_column)
w_2 = train_logistic_model(tX_2_exp, y_2, train_ratio, 1e-5, 0, 1000)#train_ridge_model(tX_2_exp, y_2, train_ratio, 1e-3)#

Performing polynomial expansion up to degree 1
Matrix has now 126 features
Gradient Descent(0/999): loss=45254.19312439771
Gradient Descent(100/999): loss=29804.21136727084
Gradient Descent(200/999): loss=28152.29942212304
Gradient Descent(300/999): loss=27782.11255514223
Gradient Descent(400/999): loss=27559.525412432777
Gradient Descent(500/999): loss=27406.437432113366
Gradient Descent(600/999): loss=27294.011954383517
Gradient Descent(700/999): loss=27207.544944503818
Gradient Descent(800/999): loss=27138.7219482441
Gradient Descent(900/999): loss=27082.486932488697
Regularized logisitc regression got score of 0.8209510682288077, loss of 27035.58640590495


## Generate predictions and save ouput in csv format for submission:

In [37]:
from data_processing import prepare_for_testing

In [38]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test, _ = load_csv_data(DATA_TEST_PATH)
tX_test = tX_test[:, to_keep]
tX_test[tX_test == -999] = np.nan

In [39]:
def predict_testset(x, degrees, w):
    x = expand_features(x, degrees, print_=False, jet_col=jet_column)
    x, _ = prepare_for_testing(x,None, logistic=True)
    y_pred = predict_labels(w, x)
    
    return y_pred

In [40]:
tX_0_test, ids_0, tX_1_test, ids_1, tX_2_test, ids_2= split_dataset(tX_test, ids_test, jet_col=jet_column) # Split into each category

In [197]:
y_pred_0 = predict_testset(tX_0_test, degree_0, w_0)


In [198]:
y_pred_1 = predict_testset(tX_1_test, degree_1, w_1)


In [199]:
y_pred_2 = predict_testset(tX_2_test, degree_2, w_2)

In [200]:
ids_test = np.concatenate([ids_0, ids_1, ids_2])
y_pred = np.concatenate([y_pred_0, y_pred_1, y_pred_2])

In [201]:
assert y_pred.shape[0] == tX_test.shape[0]

In [202]:
OUTPUT_PATH = 'submission.csv' # TODO: fill in desired name of output file for submission
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [203]:
import pandas as pd

In [204]:
labels = pd.read_csv('../data/solution-with-features.csv')[['Id', 'Prediction']]
pred = pd.read_csv('submission.csv')

In [205]:
c = pred.merge(labels, on='Id')

In [206]:
len(c[c['Prediction_x'] == c['Prediction_y']]) / len(pred)

0.8190177355263112